In [1]:
#import data
import csv
import pandas as pd

df_train=pd.read_csv('../data/01_train/train.tsv', sep='\t',
            header=0,
            names = ["Nr","Wort","Typ1","Typ2"],
            quoting=csv.QUOTE_NONE                     ,
                     keep_default_na=False,
                     encoding='utf-8')

df_train = df_train.drop(df_train[df_train['Nr']=='#'].index)
df_train=df_train.reset_index(drop=False)




In [2]:
'Label dict filter'
outside =0
label_dict = {
    'B-LOC': 2,
    'I-LOC': 3,
    'B-PER': 4,
    'I-PER': 5,
    'B-ORG': 6,
    'I-ORG': 7,
    'B-LOCderiv': outside,
    "B-LOCpart": outside,
    'B-ORGderiv': outside,
    'B-ORGpart': outside,
    'B-OTH': outside,
    'B-OTHderiv': outside,
    'B-OTHpart': outside,
    'B-PERderiv': outside,
    'B-PERpart': outside,
    'I-LOCderiv': outside,
    'I-LOCpart': outside,
    'I-ORGpart': outside,
    'I-OTH': outside,
    'I-OTHderiv': outside,
    'I-OTHpart': outside,
    'I-PERderiv': outside,
    'I-PERpart': outside,
    'O': outside
}



tokens=[]
labels=[]
ids=[]

sentence_token=[]
sentence_labels=[]

i=0

id=0
for j in range(0, len(df_train)):
    if i > 0 and df_train.Nr[j] == '1':
        tokens.append(sentence_token)
        labels.append(sentence_labels)

        sentence_token, sentence_labels = [], []
        ids.append(id)
        id = id + 1

    sentence_token.append(str(df_train.Wort[j]))
    # sentence_labels.append(df_train.Typ1[j])
    sentence_labels.append(label_dict.get(df_train.Typ1[j]))

    i = i + 1

In [3]:
'Data Preprocessing'


'Data Preprocessing'

In [4]:
'label alignment'

'label alignment'

In [5]:
from datasets.dataset_dict import DatasetDict
from datasets import Dataset


conll = {'train':Dataset.from_dict({'id':ids,'tokens':tokens,'ner_tags':labels})
    ,'validation':Dataset.from_dict({'id':ids[10:11],'tokens':tokens[10:11],'ner_tags':labels[10:11]})
     #'test':Dataset.from_dict({'label':y_test,'text':x_test})
     }

In [6]:
# from datasets import Dataset, ClassLabel, Sequence, Features, Value
#
# d = { 'ner_tags': labels, 'tokens': tokens}
# # define number of tags and their names
# tags = ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'])
# # create dataset
# ds ={'train':Dataset.from_dict(mapping=d, features=Features({"ner_tags":Sequence(tags),  'tokens': Sequence(feature=Value(dtype='string'))}))}
# # access ClassLabel feature - 0 returns tag "O", 1 returns "B-PER" etc
# ds['train'].features["ner_tags"].feature.int2str(0)
# #conll=ds

In [7]:
#test
print(conll['train'][0]['ner_tags'])
print(conll['train'][0]['tokens'])

# print(conll['train'][1]['ner_tags'])
# print(conll['train'][1]['tokens'])
#
# print(conll['train'][2]['ner_tags'])
# print(conll['train'][2]['tokens'])
#
# print(conll['train'][3]['ner_tags'])
# print(conll['train'][3]['tokens'])
#
# print(conll['train'][4]['ner_tags'])
# print(conll['train'][4]['tokens'])
#
# print(conll['train'].features['ner_tags'])




[4, 0, 0, 0, 6, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['Schartau', 'sagte', 'dem', '"', 'Tagesspiegel', '"', 'vom', 'Freitag', ',', 'Fischer', 'sei', '"', 'in', 'einer', 'Weise', 'aufgetreten', ',', 'die', 'alles', 'andere', 'als', 'überzeugend', 'war', '"', '.']


In [8]:
from transformers import AutoTokenizer

'inferenz'
model_name = 'deepset/gbert-base'
# model_name='jplu/tf-xlm-r-ner-40-lang'
# model_name='Davlan/bert-base-multilingual-cased-ner-hrl'
tokenizer = AutoTokenizer.from_pretrained(model_name)
inputs = tokenizer(conll['train'][0]['tokens'], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'Schar',
 '##ta',
 '##u',
 'sagte',
 'dem',
 '"',
 'Tages',
 '##spiegel',
 '"',
 'vom',
 'Freitag',
 ',',
 'Fischer',
 'sei',
 '"',
 'in',
 'einer',
 'Weise',
 'aufgetreten',
 ',',
 'die',
 'alles',
 'andere',
 'als',
 'überzeugen',
 '##d',
 'war',
 '"',
 '.',
 '[SEP]']

In [9]:


def tokenize_and_align_labels(sequence_batch):
    # tokenize pre-tokenized sequences
    # long sequences will be truncated to respect the maximum token length of the language model (usually 512)
    tokenized_sequences = tokenizer(sequence_batch['tokens'], truncation=True, is_split_into_words=True)

    labels = []
    # iterate over pre-tokenized tokens of single sequences
    for i, label in enumerate(sequence_batch['ner_tags']):
        # get associated word ids of the subtokens
        # if a token has multiple subtokens, then each subtoken is associated with the token's word id
        word_ids = tokenized_sequences.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        # iterate over subtokens
        for word_idx in word_ids:
            # special tokens (e.g. [CLS], [SEP]) get label id -100 -> loss function will ignore them
            if word_idx is None:
                label_ids.append(-100)
            # if the first subtoken of the next token is encountered, then associate the token's ner label with the subtoken
            # FIXME what if two consecutive tokens are identical (e.g. "is this a really really bad?")?
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # consecutive subtokens will be ignored
            else:
                label_ids.append(-100)
            # memorize the current word
            previous_word_idx = word_idx
        # add labels of the current sequence to the list of labels of the batch
        labels.append(label_ids)

    # update batch labels
    tokenized_sequences['labels'] = labels
    return tokenized_sequences

In [10]:
# # test
# example_ner_tag=conll['train'][0]['ner_tags']
# print(example_ner_tag)
# exampletokens=conll['train'][0]['tokens']
# print(exampletokens)
# example={'tokens':exampletokens,'ner_tags':example_ner_tag}
# emaple_prepocessed=tokenize_and_align_labels(example)

In [11]:
from numpy import unique

unique(df_train.Typ1)

array(['B-LOC', 'B-LOCderiv', 'B-LOCpart', 'B-ORG', 'B-ORGderiv',
       'B-ORGpart', 'B-OTH', 'B-OTHderiv', 'B-OTHpart', 'B-PER',
       'B-PERderiv', 'B-PERpart', 'I-LOC', 'I-LOCderiv', 'I-LOCpart',
       'I-ORG', 'I-ORGpart', 'I-OTH', 'I-OTHderiv', 'I-OTHpart', 'I-PER',
       'I-PERderiv', 'I-PERpart', 'O'], dtype=object)

In [12]:
# tf_train_set = model.prepare_tf_dataset(
#     conll['train'].map(tokenize_and_align_labels, batched=True),
#     shuffle=True,
#     batch_size=1,
#     collate_fn=data_collator,
# )

In [13]:
#conll['train'][0]['pos_tags']

In [14]:
#conll['train'].features['pos_tags']


In [29]:
from keras.losses import BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy, Hinge
from transformers import TFAutoModelForTokenClassification, create_optimizer

'LMtraining'

model = TFAutoModelForTokenClassification.from_pretrained(model_name, num_labels=8)
batch_size = 32
num_train_epochs = 30
num_train_steps = (len(conll['train']) // batch_size) * num_train_epochs
# optimizer, lr_schedule = create_optimizer(
#     init_lr=2e-5,
#     num_train_steps=num_train_steps,
#     weight_decay_rate=0.01,
#     num_warmup_steps=0,
# )

model.get_layer('bert').trainable=False
#SparseCategoricalCrossentropy(from_logits=True)


All model checkpoint layers were used when initializing TFBertForTokenClassification.

Some layers of TFBertForTokenClassification were not initialized from the model checkpoint at deepset/gbert-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
model.compile(optimizer="adam",  metrics=["accuracy"])
#model.summary()

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [31]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")
tf_train_set = model.prepare_tf_dataset(
    conll['train'].map(tokenize_and_align_labels, batched=True),
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    conll['validation'].map(tokenize_and_align_labels, batched=True),
    shuffle=True,
    batch_size=1,
    collate_fn=data_collator,
)

  0%|          | 0/24 [00:00<?, ?ba/s]

C:\Users\Timo\PycharmProjects\shared-tasks-wintersemester-2022-23\venv\lib\site-packages\transformers\tokenization_utils_base.py:715: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


  0%|          | 0/1 [00:00<?, ?ba/s]

In [37]:
import numpy as np


def compute_metrics(p):
    import evaluate
    label_list = [0,1,2,3,4,5,6,7]
#    label_list = tf_train_set.features[f"ner_tags"].feature.names

    seqeval = evaluate.load("seqeval")

    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


from transformers.keras_callbacks import KerasMetricCallback
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)
model.compile(optimizer="adam",  # loss=keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
#model.summary()
import os
os.environ["XLA_FLAGS"]="--xla_gpu_cuda_data_dir=C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.8"
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=30,callbacks=[metric_callback])

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


Epoch 1/30


C:\Users\Timo\PycharmProjects\shared-tasks-wintersemester-2022-23\venv\lib\site-packages\transformers\tokenization_utils_base.py:715: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


750/750 [==============================] - ETA: 0s - loss: 0.0668 - accuracy: 0.3459

ValueError: Predictions and/or references don't match the expected format.
Expected format: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')},
Input predictions: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
Input references: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:

for example in tf_train_set:
   ainput_ids= example[0]['input_ids'].numpy()
   atoken_type_ids= example[0]['token_type_ids'].numpy()
   aattention_mask= example[0]['attention_mask'].numpy()
   alabels=example[1].numpy()
   print(example[1])
   break


In [32]:
import os
os.environ["XLA_FLAGS"]="--xla_gpu_cuda_data_dir=C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.8"
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=1)

C:\Users\Timo\PycharmProjects\shared-tasks-wintersemester-2022-23\venv\lib\site-packages\transformers\tokenization_utils_base.py:715: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


750/750 [==============================] - 67s 79ms/step - loss: 0.1139 - accuracy: 0.3421 - val_loss: 9.1680e-04 - val_accuracy: 0.6207


In [ ]:
'detokenizer'


In [33]:
# test prediction
pred=model.predict(tf_train_set)


750/750 [==============================] - 55s 71ms/step


In [34]:
te
compute_metrics(pred)

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
list=[]
for a in tf_train_set:
    list.append(a)

In [ ]:
a=example[0]['input_ids'][0]

tokenizer.decode(a)

In [ ]:
from numpy import argmax

id = 2

pred_label=[]
pred_sentence=[]
for pred_logits in pred.logits[id]:
    pred_sentence.append(argmax(pred_logits))

print(pred_sentence)
print(list[id][1][0].numpy())

In [ ]:
id=1

